In [100]:
pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 718.9 kB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp38-cp38-macosx_10_9_x86_64.whl size=764908 sha256=843912f1f0499c938b340033a2b8c6b84abd9ad092762eb4afe85fb5acdda9c6
  Stored in directory: /Users/oscar/Library/Caches/pip/wheels/20/91/57/2965d4cff1b8ac7ed1b6fa25741882af3974b54a31759e10b6
Successfully built scikit-surprise
Note: you may need to restart the kernel to use updated packages.


In [343]:
pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.1/310.1 KB 1.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.16-cp38-cp38-macosx_10_9_x86_64.whl size=446306 sha256=abd931682b4bb30a628ce66d7ab0f8d2d1f5b2510c21fe1988347865106bdc9b
  Stored in directory: /Users/oscar/Library/Caches/pip/wheels/ec/bb/51/9c487d021c1373b691d13cadca0b65b6852627b1f3f43550fa
Successfully built lightfm
Note: you may need to restart the kernel to use updated packages.


In [159]:
import als_recommender

import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import ndcg_score
from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import GridSearchCV, train_test_split, cross_validate
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split as train_test_split_sk
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import ttest_ind, levene
pd.set_option('display.max_columns', None)

DATAPATH = 'archive/'

# 1. Выбор метрики

тк в наборе данных присутствуют оценки по шкале от 0.5 до 5, то в качестве итоговой метрики можно использовать NDCG@k=5 – мера качества ранжирования. Из-за большого размера датасета, я сделаю срез по фильмам и отберу 30 фильмов. Соотвественно рекомендовать более 30 фильмов не получится, я хочу попадать в топ 5.

In [4]:
movie = pd.read_csv(DATAPATH + '/movie.csv')
rating = pd.read_csv(DATAPATH + '/rating.csv', parse_dates=['timestamp'])
scores = pd.read_csv(DATAPATH + '/genome_scores.csv')
df = movie.merge(rating, how="left", on="movieId")
df = df.drop(columns='genres')
df.head()

,movieId,title,userId,rating,timestamp
0,1,Toy Story (1995),3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),11.0,4.5,2009-01-02 01:13:41


In [6]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000797 entries, 0 to 20000796
Data columns (total 5 columns):
 #   Column     Dtype         
---  ------     -----         
 0   movieId    int64         
 1   title      object        
 2   userId     float64       
 3   rating     float64       
 4   timestamp  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 2.3 GB


### Удаление дубликатов

In [7]:
df = df.sort_values('timestamp')
df.shape

(20000797, 5)

In [8]:
df = df[~(df.userId.isna() & df.rating.isna())]
df.shape

(20000263, 5)

In [9]:
df.drop_duplicates(['userId', 'movieId'], keep='last', inplace=True)
df.shape

(20000263, 5)

In [10]:
df['userId'] = df['userId'].astype(int)

Тк набор данных слишком большой, сделаю срез данных, для демонстрации пайплайна

In [11]:
movie_ids = list(df.groupby('movieId').agg('size').sort_values(ascending=False)[:30].index)
movies = list(df[['movieId', 'title']][df.movieId.isin(movie_ids)].drop_duplicates()['title'].values)
movie_ids_dict = {j: i for i, j in zip(movie_ids, movies)}

In [12]:
sample_df = df[df.movieId.isin(movie_ids)]
print(sample_df.shape)
sample_df.head()

(1474519, 5)


,movieId,title,userId,rating,timestamp
567014,47,Seven (a.k.a. Se7en) (1995),131160,5.0,1995-01-09 11:46:49
616526,50,"Usual Suspects, The (1995)",99851,5.0,1996-01-29 00:00:00
611627,50,"Usual Suspects, The (1995)",85252,5.0,1996-01-29 00:00:00
398530,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),99851,5.0,1996-01-29 00:00:00
372832,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),20821,5.0,1996-01-29 00:00:00


In [13]:
# создаю юзер-итем датафрейм
user_movie_df = sample_df.pivot_table(index=["userId"], columns=["movieId"], values="rating")
#user_movie_df = df.pivot_table(index=["userId"], columns=["title"], values="rating")
user_movie_df.head()


movieId,1,32,47,50,110,150,260,296,318,356,377,380,457,480,527,588,589,590,592,593,608,780,858,1196,1198,1210,1270,2571,2858,2959
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.5,3.5,3.5,NaN,NaN,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,3.5,NaN,NaN,NaN,4.5,4.5,NaN,NaN,NaN,NaN,4.0
2,NaN,NaN,NaN,NaN,4.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,5.0,5.0,NaN,3.0,NaN
3,4.0,4.0,NaN,5.0,NaN,NaN,5.0,NaN,5.0,NaN,NaN,NaN,5.0,5.0,NaN,NaN,4.0,NaN,NaN,5.0,NaN,3.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN
4,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,3.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,4.0,5.0,5.0,NaN,5.0,NaN,5.0,5.0,5.0,4.0,NaN,5.0,5.0,5.0,NaN,3.0,3.0,5.0,NaN,5.0,5.0,5.0,NaN,NaN,NaN,NaN


In [14]:
train_df, test_df = train_test_split_sk(
    sample_df[['userId', 'movieId', 'rating']],
    test_size=0.25,
    random_state=1
)

In [15]:
reader = Reader(rating_scale=(0.5, 5))

# Делаю набор данных 
data = Dataset.load_from_df(sample_df[['userId', 'movieId', 'rating']], reader)
trainset = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], reader).build_full_trainset()
testset = test_df[['userId', 'movieId', 'rating']].values.tolist()
type(data)

surprise.dataset.DatasetAutoFolds

# Modeling

In [16]:
sample_df

,movieId,title,userId,rating,timestamp
567014,47,Seven (a.k.a. Se7en) (1995),131160,5.0,1995-01-09 11:46:49
616526,50,"Usual Suspects, The (1995)",99851,5.0,1996-01-29 00:00:00
611627,50,"Usual Suspects, The (1995)",85252,5.0,1996-01-29 00:00:00
398530,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),99851,5.0,1996-01-29 00:00:00
372832,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),20821,5.0,1996-01-29 00:00:00
...,...,...,...,...,...
2178162,318,"Shawshank Redemption, The (1994)",107073,3.0,2015-03-31 04:45:21
4930728,858,"Godfather, The (1972)",107073,5.0,2015-03-31 04:45:23
618946,50,"Usual Suspects, The (1995)",107073,2.5,2015-03-31 04:45:24
3518355,527,Schindler's List (1993),107073,5.0,2015-03-31 04:46:43


Для валидации модели разделим выборку на обучающий и тренировочный датасет на перемешанных данных. Таким образом в тестовой выборке окажутся как новые пользователи, так и пользователи которых часть оценок оказалась в обучающей выборке. Тк нет исходно бизнес требований о том, какая ситуация нам важнее – предсказывать оценки фильмов новым пользователям, либо предсказывать оценки непросмотренных фильмов для старых пользователей, то такая схема валидации вполне позволит оценить "среднюю" ситуацию.

In [17]:
# выделяю обучающий и тестовый набор данных и обучаю 
svd_model = SVD()
svd_model.fit(trainset)

In [18]:
predictions = svd_model.test(testset)

In [19]:
accuracy.rmse(predictions)

RMSE: 0.8173


0.8173182536358711

In [20]:
cross_validate(svd_model, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=-1, verbose=True)
user_movie_df.head()

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8134  0.8125  0.8155  0.8101  0.8142  0.8131  0.0018  
MAE (testset)     0.6311  0.6290  0.6302  0.6277  0.6307  0.6298  0.0012  
Fit time          74.17   73.89   73.88   73.67   73.51   73.82   0.22    
Test time         2.42    2.38    2.31    2.29    2.40    2.36    0.05    


movieId,1,32,47,50,110,150,260,296,318,356,377,380,457,480,527,588,589,590,592,593,608,780,858,1196,1198,1210,1270,2571,2858,2959
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.5,3.5,3.5,NaN,NaN,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,3.5,NaN,NaN,NaN,4.5,4.5,NaN,NaN,NaN,NaN,4.0
2,NaN,NaN,NaN,NaN,4.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,5.0,5.0,NaN,3.0,NaN
3,4.0,4.0,NaN,5.0,NaN,NaN,5.0,NaN,5.0,NaN,NaN,NaN,5.0,5.0,NaN,NaN,4.0,NaN,NaN,5.0,NaN,3.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN
4,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,3.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,4.0,5.0,5.0,NaN,5.0,NaN,5.0,5.0,5.0,4.0,NaN,5.0,5.0,5.0,NaN,3.0,3.0,5.0,NaN,5.0,5.0,5.0,NaN,NaN,NaN,NaN


In [21]:
# Делаем предсказание для фильма который был оценен
print('Fight Club (1999)')
svd_model.predict(uid=1, iid=movie_ids_dict['Fight Club (1999)'], verbose=True)

Fight Club (1999)
user: 1          item: 2959       r_ui = None   est = 3.83   {'was_impossible': False}


Prediction(uid=1, iid=2959, r_ui=None, est=3.8298334321831544, details={'was_impossible': False})

In [22]:
# Делаем предсказание для фильма который не был оценен
print('Aladdin (1992)')
svd_model.predict(uid=1, iid=movie_ids_dict['Aladdin (1992)'], verbose=True)

Aladdin (1992)
user: 1          item: 590        r_ui = None   est = 3.63   {'was_impossible': False}


Prediction(uid=1, iid=590, r_ui=None, est=3.6341502702419586, details={'was_impossible': False})

## Подбор гиперпараметров

In [23]:
# here it is our parameters for the model, epoch and learning rate
param_grid = {'n_epochs': [5, 12, 30, 50], 
              'lr_all': [0.002, 0.005],
             'n_factors':[2, 10]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=-1, joblib_verbose=True)

gs.fit(data)

gs.best_score['rmse']

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  3.4min finished


0.8164638454610195

In [24]:
gs.best_params

{'rmse': {'n_epochs': 30, 'lr_all': 0.005, 'n_factors': 10},
 'mae': {'n_epochs': 50, 'lr_all': 0.005, 'n_factors': 10}}

In [25]:
# Обучаем итоговую модель 
svd_model = SVD(**gs.best_params['rmse'])

svd_model.fit(trainset)

In [26]:
predictions = svd_model.test(testset)

In [27]:
accuracy.rmse(predictions)

RMSE: 0.8100


0.8100485870579205

In [28]:
predictions_svd_df = pd.DataFrame(
    [(pred.uid, pred.iid, pred.r_ui, pred.est)for pred in predictions],
    columns=['userId', 'movieId', 'rating', 'predictRating'])
predictions_svd_df

,userId,movieId,rating,predictRating
0,18421.0,1196.0,4.5,4.237327
1,14276.0,589.0,5.0,4.278268
2,46194.0,480.0,4.0,4.180207
3,126698.0,480.0,3.5,3.915814
4,107886.0,589.0,4.5,4.208105
...,...,...,...,...
368625,46928.0,780.0,1.0,2.730767
368626,99455.0,110.0,3.5,4.301092
368627,87467.0,589.0,5.0,4.363435
368628,75275.0,110.0,5.0,2.909581


In [146]:
ndcg_scores_svd = []
for i, tmp in tqdm(predictions_svd_df.groupby('userId')):
    try:
        ndcg_scores_svd.append(ndcg_score([tmp['rating']], 
                                  [tmp['predictRating']],
                                  k=5)
                      )
    except:
        pass

100%|██████████| 109004/109004 [00:48<00:00, 2251.14it/s]


In [33]:
print(f"average NDCG@k per user = {np.mean(ndcg_scores)}")

average NDCG@k per user = 0.9687893790178088


# LightFM

В данном разделе я попробую обогатить модель не только оценками от юзеров, но также дополнительной информацией по фильмам. 

In [34]:
from lightfm import LightFM

/Users/oscar/opt/anaconda3/envs/python38_main/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [35]:
# В качестве фичей фильмов можно использовать релеватность данных им тегов
item_features = scores[scores.movieId.isin(movie_ids)].pivot_table(index='movieId', columns='tagId', values='relevance')
item_features.head()

tagId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1

In [36]:
lfm = LightFM(loss='warp')

In [58]:
train_df_lfm = user_movie_df.copy()
for p in tqdm(test_df[['userId', 'movieId']].itertuples()):
    train_df_lfm.loc[p.userId, p.movieId] = None

368630it [02:28, 2482.33it/s]


In [142]:
train_df_csr = csr_matrix(train_df_lfm.fillna(0))
item_features = csr_matrix(item_features)

In [63]:
lfm.fit(
    train_df_csr,
    item_features=item_features,
    verbose=True,
    num_threads=6,
    epochs=2
)

Epoch: 100%|██████████| 2/2 [05:11<00:00, 155.99s/it]


In [90]:
uid_dict = {j: i for i, j in enumerate(train_df_lfm.index)}
iid_dict = {j: i for i, j in enumerate(train_df_lfm.columns)}

In [129]:
lfm_predicted = lfm.predict(
    user_ids=test_df['userId'].map(uid_dict).values,
    item_ids=test_df['movieId'].map(iid_dict).values,
    item_features=item_features
)

In [130]:
mms = MinMaxScaler((0.5, 5))
lfm_predicted = mms.fit_transform(lfm_predicted.reshape(-1, 1)).ravel()

In [135]:
predictions_lfm_df = test_df.copy()
predictions_lfm_df['predictRating'] = lfm_predicted

In [140]:
def ncg_scores(predictions):
    ndcg_scores = []
    for i, tmp in tqdm(predictions.groupby('userId')):
        try:
            ndcg_scores.append(ndcg_score(
                                        [tmp['rating']], 
                                        [tmp['predictRating']],
                                        k=5
                                    )
                          )
        except:
            pass
        
    return ndcg_scores

ndcg_scores = ncg_scores(predictions_lfm_df)

100%|██████████| 109004/109004 [00:52<00:00, 2090.33it/s]


In [138]:
print(f"average NDCG@k per user = {np.mean(ndcg_scores)}")

average NDCG@k per user = 0.9510914620155546


In [139]:
LightFM(loss='warp')

In [144]:
scores = []
models = []
for no_components in [2, 5, 10]:
    for learning_schedule in ['adagrad', 'adadelta']:
        print(no_components, learning_schedule)
        lfm = LightFM(loss='warp',
                      no_components=no_components,
                      learning_schedule=learning_schedule
                )
        
        lfm.fit(
            train_df_csr,
            item_features=item_features,
            verbose=True,
            num_threads=6,
            epochs=2,
        )
        
        lfm_predicted = lfm.predict(
            user_ids=test_df['userId'].map(uid_dict).values,
            item_ids=test_df['movieId'].map(iid_dict).values,
            item_features=item_features
        )
        
        lfm_predicted = mms.fit_transform(lfm_predicted.reshape(-1, 1)).ravel()
        
        predictions_lfm_df['predictRating'] = lfm_predicted
        ndcg_scores = ncg_scores(predictions_lfm_df)
        scores.append(np.mean(ndcg_scores))
        models.append(lfm)
        print(f'score = {scores[-1]}')

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

2 adagrad


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

score = 0.9517622091445416
2 adadelta


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

...

score = 0.9569931321363286
10 adagrad


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

score = 0.9508850238053879
10 adadelta


100%|██████████| 109004/109004 [00:48<00:00, 2268.05it/s]

score = 0.9581680259015034


In [147]:
best_ind = np.argmax(scores)
lfm_predicted = models[best_ind].predict(
            user_ids=test_df['userId'].map(uid_dict).values,
            item_ids=test_df['movieId'].map(iid_dict).values,
            item_features=item_features
        )
        
lfm_predicted = mms.fit_transform(lfm_predicted.reshape(-1, 1)).ravel()
        
predictions_lfm_df['predictRating'] = lfm_predicted
ndcg_scores_lfm = ncg_scores(predictions_lfm_df)

100%|██████████| 109004/109004 [00:47<00:00, 2276.44it/s]


# Результаты

Проверим тестов Стъюдента равенство средних скоров двух выборок, полученных двумя моделями. Сначала проверим равенство дисперсий для верного определения параметра.

In [161]:
np.var(ndcg_scores_svd)

0.0021912566135901825

In [162]:
np.var(ndcg_scores_lfm)

0.0032600506701803422

In [160]:
levene(ndcg_scores_svd, ndcg_scores_lfm)

LeveneResult(statistic=1501.9359916752023, pvalue=0.0)

Дисперсии не равны, это подтвердил тест Левена

In [164]:
ttest_ind(ndcg_scores_svd, ndcg_scores_lfm, equal_var=False)

Ttest_indResult(statistic=41.48907503556589, pvalue=0.0)

Получаем статистически значимый результат, SVD дал лучшую метрику.

In [155]:
pd.DataFrame({
    'SVD': [np.mean(ndcg_scores_svd)],
    'lightFM': [np.mean(ndcg_scores_lfm)]
}, index=['NDCG'])

,SVD,lightFM
NDCG,0.968789,0.958168
